[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/twMr7/Python-Machine-Learning/blob/master/09-Other_Utilities.ipynb)

# 9. 其他實用工具 Other Utilities

Python 的通用性來自於豐富的標準函式庫，本章介紹以下幾種常用的工具模組。
+ [**9.1 日期與時間（Date and Time）**](#module-datetime)
+ [**9.2 物件序列化（Python Object Serialization）**](#module-pickle)
+ [**9.3 亂數（Random Numbers）**](#module-random)
+ [**9.4 數學函數（Math Functions）**](#module-math)
+ [**9.5 檔案系統路徑（File System Paths）**](#module-pathlib)


<a id="module-datetime"></a>

## 9.1 日期與時間 Date and Time

Python 標準函式庫中的 [`datetime`](https://docs.python.org/3/library/datetime.html#datetime-objects) 模組可以用來處理日期時間相關的資料，包含了 `date`, `time`, `datetime`, `timedelta`, `timezone` 等型別。

In [1]:
# 載入 datetime 模組
from datetime import datetime

In [2]:
# 現在的日期時間，返回 datetime 型別
t1 = datetime.now()
# 轉成字串 str 型別
str(t1)

'2019-08-09 12:04:30.619654'

In [3]:
# 從日期時間的字串轉成 datetime 型別
tformat = '%Y-%m-%d %H:%M:%S'
t2 = datetime.strptime('2018-10-28  15:10:00', tformat)

# 比較兩個 datetime
if (t1 > t2):
    print('t1 比 t2 晚', t1 - t2)
else:
    print('t2 比 t1 晚', t2 - t1)

t1 比 t2 晚 284 days, 20:54:30.619654


<a id="module-pickle"></a>

## 9.2 物件序列化 Python Object Serialization

Python 標準函式庫中的 [`pickle`](https://docs.python.org/3/library/pickle.html#module-pickle) 模組，提供了將 Python 物件序列化（serializing）及解序列化（de-serializing）的方法。 序列化指的是將物件階層轉換成位元組串流（byte stream），以方便物件的儲存、網路傳送、以及不同平臺的互通交換，反向的解序列化操作則是將位元組串流轉換成物件階層。

+ `pickle` 模組可以將物件儲存至檔案，或從檔案載入物件，檔案的存取需使用 binary 模式。
+ `pickle` 模組提供的序列化功能只適用於 Python 物件專用，標準函式庫中另外有跨平臺及程式語言的通用型的序列化模組 [`json`](https://docs.python.org/3/library/json.html#module-json)，但 `json` 只支援較少的 Python 內建物件型別。

In [4]:
# 載入 pickle 模組
import pickle

In [5]:
# 建立一個數據記錄的結構
record = [
    {'時間':datetime.strptime('2019-04-03 10:35:58', tformat), '體溫':37.0, '速度':35.0, '心率':92},
    {'時間':datetime.strptime('2019-04-03 10:37:00', tformat), '體溫':37.1, '速度':33.8, '心率':97},
    {'時間':datetime.strptime('2019-04-03 10:37:59', tformat), '體溫':37.4, '速度':35.5, '心率':99}
]

In [6]:
# 開啟新的 binary 檔案，用 pickle 將 record 物件 serialize
pfile = open('record.pkl', 'wb')
pickle.dump(record, pfile)
pfile.close()

In [7]:
# 讀入檔案，將 record 物件 de-serialize
pfile = open('record.pkl', 'rb')
record2 = pickle.load(pfile)
pfile.close()
record2

[{'時間': datetime.datetime(2019, 4, 3, 10, 35, 58),
  '體溫': 37.0,
  '速度': 35.0,
  '心率': 92},
 {'時間': datetime.datetime(2019, 4, 3, 10, 37),
  '體溫': 37.1,
  '速度': 33.8,
  '心率': 97},
 {'時間': datetime.datetime(2019, 4, 3, 10, 37, 59),
  '體溫': 37.4,
  '速度': 35.5,
  '心率': 99}]

<a id="module-random"></a>

## 9.3 亂數 Random Numbers

Python 標準函式庫中的 [`random`](https://docs.python.org/3/library/random.html) 模組，提供了擬隨機（pseudo-random）亂數產生的方法。

+ `random()` - 返回下一個 [0.0, 1.0) 區間內的隨機實數。
+ `randrange(start, stop[, step])` - 返回下一個 [start, stop) 區間內的隨機整數。
+ `randint(a, b)` - 返回下一個 [a, b] 區間內的隨機整數，同 `randrange(a, b+1)`。
+ `choice(seq)` - 從 seq 序列中隨機選取其中一個成員。
+ `shuffle(seq)` - 將 seq 序列中的元素順序重新隨機排列，序列必須是可就地變更的容器類別。
+ `sample(seq, k)` - 從 seq 序列或集合中，返回隨機選取 k 個樣本的 List 清單。

In [8]:
# 載入 random 模組
import random

In [9]:
# 產生 100 個隨機實數數列
Lr = [random.random() for x in range(100)]
print(Lr)

[0.7052357497074248, 0.6323705607249648, 0.08389069399178051, 0.10300617244038568, 0.7719857266399388, 0.23469159702607745, 0.4158002043740896, 0.16533992359494576, 0.31211970489083973, 0.7094583094731348, 0.0471261784692667, 0.4125856145957225, 0.2392153600939767, 0.4347507013143791, 0.9278241200604752, 0.24833079688828374, 0.33123958082531935, 0.32507672334822635, 0.8526145803912517, 0.20500609323054886, 0.0284189001974402, 0.4622914800150507, 0.08734510178725685, 0.32995209433260475, 0.17651039828766713, 0.07513472495449924, 0.4050043725860635, 0.41637072449580836, 0.282858062912293, 0.1246876140181834, 0.17288844179400942, 0.15836109303591261, 0.5180233823181128, 0.39929761222585824, 0.2292315979370424, 0.1396097067634623, 0.8106270317270603, 0.7129558050659258, 0.7422278342690081, 0.8211036941296195, 0.0466747994828931, 0.2341192230934801, 0.8961424220714813, 0.9888276289370502, 0.22291323154038722, 0.31815887772203233, 0.18338677766015077, 0.5757711158247317, 0.9324605249624675, 

In [10]:
# 產生 100 個隨機整數數列
Li = [random.randint(1, 100) for x in range(100)]
print(Li)

[51, 69, 27, 34, 37, 53, 26, 4, 35, 91, 36, 65, 80, 94, 67, 77, 37, 74, 82, 16, 89, 97, 76, 16, 57, 30, 54, 46, 11, 57, 89, 89, 42, 26, 19, 99, 23, 4, 47, 100, 66, 81, 91, 65, 39, 12, 54, 4, 42, 1, 74, 6, 29, 33, 86, 40, 34, 53, 75, 1, 54, 17, 43, 43, 92, 38, 64, 34, 37, 100, 99, 11, 69, 55, 92, 37, 68, 48, 69, 30, 10, 44, 89, 16, 74, 79, 74, 58, 29, 58, 58, 90, 16, 70, 93, 84, 88, 75, 35, 92]


In [11]:
# 從數列中隨機選取 10 個樣本，產生新的隨機數列
[x * y for x, y in zip(random.sample(Lr, 10), random.sample(Li, 10))]

[41.969517750054834,
 42.82877987544969,
 27.462429867953297,
 24.83572261008218,
 6.219599362443394,
 8.06020858303077,
 2.3929263284485978,
 8.828194191524465,
 76.08157784495897,
 21.952962562820233]

<a id="module-math"></a>

## 9.4 數學函數 Math Functions

Python 標準函式庫中的 [`math`](https://docs.python.org/3/library/math.html) 模組，提供了用於實數運算的常用函數。

In [12]:
# 載入 math 模組
import math

In [13]:
# 內建函式的 sum() 在浮點數運算的精度不足
print(sum([.1, .1, .1, .1, .1, .1, .1, .1, .1, .1]))

0.9999999999999999


In [14]:
# math 模組的 fsum() 可避免精度的誤差
print(math.fsum([.1, .1, .1, .1, .1, .1, .1, .1, .1, .1]))

1.0


In [15]:
# cosine 180 度
print('cosine(pi) =', math.cos(math.pi))

cosine(pi) = -1.0


In [16]:
# sine 90 度
print('sine(pi/2) =', math.sin(math.radians(90)))

sine(pi/2) = 1.0


<a id="module-pathlib"></a>

## 9.5 檔案系統路徑 File System Paths

Python 標準函式庫中的 [`pathlib`](https://docs.python.org/3/library/pathlib.html) 模組，提供了通用於不同平台的檔案系統路徑操作，`Path` 物件可以比較、解析路徑的組成部份、也可以串接重組，主要有以下屬性：

+ `Path.drive` - 目標路徑的磁碟代號
+ `Path.root` - 目標路徑的根目錄
+ `Path.parent` - 目標路徑的上層目錄
+ `Path.name` - 目標路徑最後部份的名字
+ `Path.suffix` - 目標路徑最後部份的副檔名
+ `Path.stem` - 目標路徑最後部份去除副檔名的名字

常用的 `Path` 類別方法如下：

+ `Path.cwd()` - 目前工作目錄。
+ `Path.home()` - 登入使用者的家目錄。
+ `Path(str)` - 從字串 str 建立路徑物件。
+ `Path.exists()` - 路徑的檔案或目錄是否存在。
+ `Path.glob(pattern)` - 返回生成函式，用來列出路徑下符合指定 pattern 的所有檔案或目錄。
+ `Path.is_dir()` - 檢查路徑的目標是否爲目錄。
+ `Path.is_file()` - 檢查路徑的目標是否爲檔案。
+ `Path.iterdir()` - 當目標路徑爲目錄時，用來迭代尋訪目錄下的所有檔案。
+ `Path.mkdir()` - 當目標路徑爲目錄時，爲該目標建立目錄。
+ `Path.rename(new_name)` - 重新命名檔案。
+ `Path.open(mode)` - 功能同內建函式 `open()`，使用指定模式開啓檔案，返回檔案物件。 


In [17]:
# 載入 Path 類別
from pathlib import Path

In [18]:
# 列出目前工作目錄下所有的檔案及目錄
pwd = Path.cwd()
print('Current working directory: ', pwd)
for f in pwd.iterdir():
    print(f.name)

Current working directory:  C:\Users\James\Documents\Code\Lecture\Python-Machine-Learning\Lecture-Notes
.git
.gitignore
.ipynb_checkpoints
01-Getting_Started.ipynb
02-Syntax_Overview_1.ipynb
03-Syntax_Overview_2.ipynb
04-String_Operations.ipynb
05-List_Operations.ipynb
06-Tuple_Operations.ipynb
07-Dict_Operations.ipynb
08-File_Operations.ipynb
09-Other_Utilities.ipynb
10-Coding_Project.ipynb
11-Numpy_Vectorized_Computation.ipynb
12-Matplotlib_Data_Visualization.ipynb
13-Pandas_Data_Processing.ipynb
14-Sklearn_Building_A_Machine_Learning_Model.ipynb
15-Sklearn_Data_Preprocessing.ipynb
16-Sklearn_Best_Practice_Techniques.ipynb
17-Artificial_Neural_Network_with_tf_Keras.ipynb
18-ANN_Case_Studies.ipynb
19-Practical_Autoencoders.ipynb
dataset
hello.txt
image
README.md
record.pkl
uhello.txt


In [19]:
# 建立一個記錄檔案名字與副檔名對照的字典
{f.stem:f.suffix for f in pwd.iterdir() if f.is_file()}

{'.gitignore': '',
 '01-Getting_Started': '.ipynb',
 '02-Syntax_Overview_1': '.ipynb',
 '03-Syntax_Overview_2': '.ipynb',
 '04-String_Operations': '.ipynb',
 '05-List_Operations': '.ipynb',
 '06-Tuple_Operations': '.ipynb',
 '07-Dict_Operations': '.ipynb',
 '08-File_Operations': '.ipynb',
 '09-Other_Utilities': '.ipynb',
 '10-Coding_Project': '.ipynb',
 '11-Numpy_Vectorized_Computation': '.ipynb',
 '12-Matplotlib_Data_Visualization': '.ipynb',
 '13-Pandas_Data_Processing': '.ipynb',
 '14-Sklearn_Building_A_Machine_Learning_Model': '.ipynb',
 '15-Sklearn_Data_Preprocessing': '.ipynb',
 '16-Sklearn_Best_Practice_Techniques': '.ipynb',
 '17-Artificial_Neural_Network_with_tf_Keras': '.ipynb',
 '18-ANN_Case_Studies': '.ipynb',
 '19-Practical_Autoencoders': '.ipynb',
 'hello': '.txt',
 'README': '.md',
 'record': '.pkl',
 'uhello': '.txt'}

In [20]:
# 列出目前工作目錄下所有副檔名是 .ipynb 的檔案
print([f.name for f in pwd.glob('*.ipynb')])

['01-Getting_Started.ipynb', '02-Syntax_Overview_1.ipynb', '03-Syntax_Overview_2.ipynb', '04-String_Operations.ipynb', '05-List_Operations.ipynb', '06-Tuple_Operations.ipynb', '07-Dict_Operations.ipynb', '08-File_Operations.ipynb', '09-Other_Utilities.ipynb', '10-Coding_Project.ipynb', '11-Numpy_Vectorized_Computation.ipynb', '12-Matplotlib_Data_Visualization.ipynb', '13-Pandas_Data_Processing.ipynb', '14-Sklearn_Building_A_Machine_Learning_Model.ipynb', '15-Sklearn_Data_Preprocessing.ipynb', '16-Sklearn_Best_Practice_Techniques.ipynb', '17-Artificial_Neural_Network_with_tf_Keras.ipynb', '18-ANN_Case_Studies.ipynb', '19-Practical_Autoencoders.ipynb']


In [21]:
# 目前工作目錄下所有副檔名是 .ipynb 的路徑清單
list(pwd.glob('*.ipynb'))

[WindowsPath('C:/Users/James/Documents/Code/Lecture/Python-Machine-Learning/Lecture-Notes/01-Getting_Started.ipynb'),
 WindowsPath('C:/Users/James/Documents/Code/Lecture/Python-Machine-Learning/Lecture-Notes/02-Syntax_Overview_1.ipynb'),
 WindowsPath('C:/Users/James/Documents/Code/Lecture/Python-Machine-Learning/Lecture-Notes/03-Syntax_Overview_2.ipynb'),
 WindowsPath('C:/Users/James/Documents/Code/Lecture/Python-Machine-Learning/Lecture-Notes/04-String_Operations.ipynb'),
 WindowsPath('C:/Users/James/Documents/Code/Lecture/Python-Machine-Learning/Lecture-Notes/05-List_Operations.ipynb'),
 WindowsPath('C:/Users/James/Documents/Code/Lecture/Python-Machine-Learning/Lecture-Notes/06-Tuple_Operations.ipynb'),
 WindowsPath('C:/Users/James/Documents/Code/Lecture/Python-Machine-Learning/Lecture-Notes/07-Dict_Operations.ipynb'),
 WindowsPath('C:/Users/James/Documents/Code/Lecture/Python-Machine-Learning/Lecture-Notes/08-File_Operations.ipynb'),
 WindowsPath('C:/Users/James/Documents/Code/Lectu